In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import ensemble
from joblib import dump, load

pd.set_option('display.max_columns', None)

In [34]:
data=pd.read_csv('./train.csv')

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 删除重复错误样本

In [35]:
dIndex = data.duplicated()
data1 = data.copy()
print(data1.shape)
data1.drop_duplicates(keep='first', inplace=True)
print(data1.shape)

(255080, 23)
(255069, 23)


In [36]:
invalid_list = data1[data1['livingRoom']=='#NAME?'].index.tolist()
data1.drop(invalid_list, inplace=True)
print(data1.shape)

(255041, 23)


## 数据处理

In [37]:
data2 = data1.copy()
data2['floorDegree'] = data2['floor'].str.extract('(\D+)')
data2['floorNumber'] = data2['floor'].str.extract('(\d+)')
data2['tradeYear'] = data2['tradeTime'].map(lambda x:x.split('-')[0])
data2['tradeMonth'] = data2['tradeTime'].map(lambda x:x.split('-')[1])
data2['tradeDay'] = data2['tradeTime'].map(lambda x:x.split('-')[2])
data2 = data2.drop(['Cid', 'floor', 'tradeTime'], axis=1)

In [38]:
data3 = data2.copy()
data3['floorDegree'][data2['floorDegree'] == '未知 '] = np.nan
data3['floorDegree'][data2['floorDegree'] == '混合结构'] = 6
data3['floorDegree'][data2['floorDegree'] == '钢混结构'] = 5
data3['floorDegree'][data2['floorDegree'] == '底 '] = 4
data3['floorDegree'][data2['floorDegree'] == '顶 '] = 3
data3['floorDegree'][data2['floorDegree'] == '高 '] = 2
data3['floorDegree'][data2['floorDegree'] == '中 '] = 1
data3['floorDegree'][data2['floorDegree'] == '低 '] = 0

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [39]:
data3['constructionTime'] = data3['constructionTime'].replace('未知', '0')
data3 = data3.replace('未知', np.nan)

## 缺失值处理及改变数据类型

In [40]:
data3.drop(columns=['DOM', 'floorDegree'],inplace=True)

In [41]:
intcols = ['livingRoom', 'drawingRoom', 'bathRoom', 'floorNumber', 'tradeYear', 'tradeMonth', 'tradeDay', 'constructionTime']
data3.loc[:,intcols] = data3.loc[:,intcols].astype('int64')

In [42]:
contime_list = data2[data2['constructionTime']!='未知'].index.tolist()
connotime_list = data2[data2['constructionTime']=='未知'].index.tolist()
delta = data3['tradeYear'][contime_list] - data3['constructionTime'][contime_list]
data3['constructionTime'][connotime_list] = data3['tradeYear'][connotime_list] - delta.mean()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [43]:
print(data3.shape)
data3.dropna(inplace=True)
print(data3.shape)

(255041, 23)
(253142, 23)


## 处理年份，用2019减去该年份

In [44]:
data3.loc[:,'constructionTime'] = 2019.0 - data3.loc[:,'constructionTime']
data3.loc[:,'tradeYear'] = 2019 - data3.loc[:,'tradeYear']

## 数据归一化

In [45]:
price_col =data3['price']
data3.drop(['price'],axis=1,inplace=True)
x = data3.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data4 = pd.DataFrame(x_scaled,columns=[column for column in data3])
data4['price'] = price_col.values.tolist()

In [47]:
print(data4.shape)
print(data4['tradeYear'].value_counts())

(253142, 23)
0.1250    71981
0.1875    55402
0.0625    34306
0.3125    30885
0.3750    29564
0.2500    25929
0.4375     4746
0.0000      179
0.5000      144
1.0000        3
0.9375        1
0.5625        1
0.6250        1
Name: tradeYear, dtype: int64


## 挑选训练数据块，2017年3个，2016年2个，2015年5个，2014年3个

In [49]:
block_size = 1000
train_data_2017 = data4[data4['tradeYear']==0.0625].sample(n=(3*block_size))
train_data_2016 = data4[data4['tradeYear']==0.125].sample(n=(2*block_size))
train_data_2015 = data4[data4['tradeYear']==0.1875].sample(n=(5*block_size))
train_data_2014 = data4[data4['tradeYear']==0.3125].sample(n=(3*block_size))

train_label_2017 = train_data_2017['price']
train_label_2016 = train_data_2016['price']
train_label_2015 = train_data_2015['price']
train_label_2014 = train_data_2014['price']

train_data_2017.drop(columns=['tradeMonth', 'tradeDay','price'], axis=1, inplace=True)
train_data_2016.drop(columns=['tradeMonth', 'tradeDay','price'], axis=1, inplace=True)
train_data_2015.drop(columns=['tradeMonth', 'tradeDay','price'], axis=1, inplace=True)
train_data_2014.drop(columns=['tradeMonth', 'tradeDay','price'], axis=1, inplace=True)

In [50]:
def addnoise(train_data_noised, epsilon, block_size, block_num):
    column_num = train_data_noised.shape[1]
    column_name = [column for column in train_data_noised]
    for i in range(block_num):
        noise = np.random.laplace(scale=(column_num/epsilon), size=(1,column_num))
        noise = (noise / block_size).tolist()
        for j in range(block_size):
            idx = train_data_noised.index[(i*block_size+j)]
            for k in range(column_num):
                item_name = column_name[k]
                train_data_noised.loc[idx,item_name] = train_data_noised.loc[idx,item_name] + noise[0][k]
    return train_data_noised

In [133]:
reg = load('BaseModel.joblib')

In [134]:
train_data1 = train_data_2015.copy()[3000:]
train_data2 = train_data_2014.copy()
train_data = pd.concat([train_data1, train_data2])
print(train_data.shape)
train_data = addnoise(train_data, 0.4, 1000, 5)
train_label1 = train_label_2015.copy()[3000:]
train_label2 = train_label_2014.copy()
train_label = pd.concat([train_label1, train_label2])
print(train_label.shape)
reg.fit(train_data, train_label)

(5000, 20)
(5000,)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=5, min_weight_fraction_leaf=0.0,
             n_estimators=400, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [135]:
pred = reg.predict(test_data_2015)
test_label = test_label_2015

In [136]:
from sklearn.metrics import mean_squared_error, median_absolute_error

print('MSE为：',mean_squared_error(pred, test_label))
print('MedAE为：',median_absolute_error(pred,test_label))

MSE为： 249679951.8529485
MedAE为： 6569.781391472923


## 挑选测试数据块

In [21]:
test_data=pd.read_csv('./test.csv')
test_price = pd.read_csv('./test_price.csv', header=None)

In [22]:
test_data2 = test_data.copy()

test_data2['tradeYear'] = test_data2['tradeTime'].map(lambda x:x.split('-')[0])
test_data2['tradeMonth'] = test_data2['tradeTime'].map(lambda x:x.split('-')[1])
test_data2['tradeDay'] = test_data2['tradeTime'].map(lambda x:x.split('-')[2])
test_data2['floorDegree'] = test_data2['floor'].str.extract('(\D+)')
test_data2['floorNumber'] = test_data2['floor'].str.extract('(\d+)')
test_data2 = test_data2.drop(['Cid', 'floor', 'tradeTime'], axis=1)
test_data2['tradeYear'].value_counts()

test_invalid = test_data2[test_data2['livingRoom']=='#NAME?'].index.tolist()
test_data2.drop(test_invalid, inplace=True)

In [23]:
test_data3 = test_data2.copy()
test_data3['floorDegree'][test_data2['floorDegree'] == '未知 '] = np.nan
test_data3['floorDegree'][test_data2['floorDegree'] == '混合结构'] = 6
test_data3['floorDegree'][test_data2['floorDegree'] == '钢混结构'] = 5
test_data3['floorDegree'][test_data2['floorDegree'] == '底 '] = 4
test_data3['floorDegree'][test_data2['floorDegree'] == '顶 '] = 3
test_data3['floorDegree'][test_data2['floorDegree'] == '高 '] = 2
test_data3['floorDegree'][test_data2['floorDegree'] == '中 '] = 1
test_data3['floorDegree'][test_data2['floorDegree'] == '低 '] = 0

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [24]:
test_data3['constructionTime'] = test_data3['constructionTime'].replace('未知', '0')
test_data3 = test_data3.replace('未知', np.nan)
test_data3.drop(columns=['DOM', 'floorDegree'],inplace=True)

In [25]:
test_data3['buildingType'] = test_data3['buildingType'].replace(np.nan, test_data3['buildingType'].mean())
test_data3['communityAverage'] = test_data3['communityAverage'].replace(np.nan, test_data3['communityAverage'].mean())

In [26]:
intcols = ['livingRoom', 'drawingRoom', 'bathRoom', 'floorNumber', 'tradeYear', 'tradeMonth', 'tradeDay', 'constructionTime']
test_data3.loc[:,intcols] = test_data3.loc[:,intcols].astype('int64')

In [27]:
test_data3.loc[:,'constructionTime'] = 2019.0 - test_data3.loc[:,'constructionTime']
test_data3.loc[:,'tradeYear'] = 2019 - test_data3.loc[:,'tradeYear']

In [28]:
test_data3.drop(columns=['tradeMonth', 'tradeDay'], axis=1, inplace=True)

In [29]:
missing = test_data3.isnull().sum()
missing

Lng                    0
Lat                    0
followers              0
square                 0
livingRoom             0
drawingRoom            0
kitchen                0
bathRoom               0
buildingType           0
constructionTime       0
renovationCondition    0
buildingStructure      0
ladderRatio            0
elevator               0
fiveYearsProperty      0
subway                 0
district               0
communityAverage       0
tradeYear              0
floorNumber            0
dtype: int64

In [30]:
from sklearn import preprocessing

y = test_data3.values
min_max_scaler = preprocessing.MinMaxScaler()
y_scaled = min_max_scaler.fit_transform(y)
test = pd.DataFrame(y_scaled,columns=[column for column in test_data3])

In [31]:
print(test['tradeYear'].value_counts())
print(test.shape)

0.250    18175
0.375    14063
0.125     8514
0.625     7684
0.750     7508
0.500     6525
0.875     1223
0.000       42
1.000       33
Name: tradeYear, dtype: int64
(63767, 20)


In [56]:
print(test_price.shape)
test_data[test_data['livingRoom']=='#NAME?']

(63771, 1)


,Lng,Lat,Cid,tradeTime,DOM,followers,square,livingRoom,drawingRoom,kitchen,bathRoom,floor,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
16231,116.238792,39.852889,1111027381738,2011-11-16,NaN,0,37.89,#NAME?,低 6,4,2004,混合结构,0.048,0,1,0,0.0,NaN,NaN,NaN,2,43567.0
18702,116.433401,40.107867,1111027378309,2015-10-24,1.0,0,90.59,#NAME?,高 6,4,2003,混合结构,0.500,0,0,0,0.0,NaN,NaN,NaN,6,41055.0
38854,116.122354,39.937384,1111027376723,2014-10-06,1.0,0,82.09,#NAME?,顶 6,4,2000,钢混结构,0.500,0,0,0,0.0,NaN,NaN,NaN,12,36358.0
55519,116.238792,39.852889,1111027381738,2011-11-16,NaN,0,37.89,#NAME?,低 6,4,2004,混合结构,0.048,0,0,0,0.0,NaN,NaN,NaN,2,43567.0


In [33]:
test_data_2016 = test[test['tradeYear']==0.250]
test_data_2015 = test[test['tradeYear']==0.375]
test_data_2014 = test[test['tradeYear']==0.625]

In [73]:
# test_price.drop(test_invalid, inplace=True)
test_label_2016 = test_price[0].values[test['tradeYear']==0.250]
test_label_2015 = test_price[0].values[test['tradeYear']==0.375]
test_label_2014 = test_price[0].values[test['tradeYear']==0.625]